In [145]:
from tensorflow.keras import layers, models
from tensorflow.keras.optimizers import Adam
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
import os
from glob import glob
import cv2
import numpy as np
from scipy.fft import fft
import pandas as pd

In [146]:
framelist = []
data_folder = 'labeled_frames_fixed'
video = './vidscapstone/vid10.mp4'

for image_path in glob(os.path.join(data_folder, '*.jpg')):
    framelist.append(image_path)

# framelist = [
#              'labeled_frames_fixed\\vid9.mp4_00054_2.jpg', 
#              'labeled_frames_fixed\\vid9.mp4_00047_0.jpg', 
#              'labeled_frames_fixed\\vid9.mp4_00053_3.jpg', 
#              'labeled_frames_fixed\\vid9.mp4_00046_2.jpg',
#              ]
    
# Load model
model = tf.keras.models.load_model('testing_binary_class_classification/testing_binary_class_classification_model_Adam.h5')

In [147]:
def extract_blinking_lights(img):

    if img is None:
        print(f"Error: Unable to load image from {image_path}")
    else:
        # Convert the image to grayscale
        grey = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        
        # Apply thresholding to highlight the lights
        _, thresholded = cv2.threshold(grey, 200, 255, cv2.THRESH_BINARY)
        
        # Find contours in the thresholded image
        contours, _ = cv2.findContours(thresholded, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        
        # Extract ROIs based on contours
        rois = []
        for contour in contours:
            x, y, w, h = cv2.boundingRect(contour)
            w = w + 20
            h = h + 20
            x = x - 10
            y = y - 10
            
            rois.append((x,y,w,h))
        
        return rois

def are_similar_coordinates(coord1, coord2):
    x1, y1, w1, h1 = coord1
    x2, y2, w2, h2 = coord2
    
    # Adjust these threshold values based on your requirements
    position_threshold = 50  # Adjust as needed
    size_threshold = 50  # Adjust as needed
    
    # Check if the coordinates are similar in terms of position and size
    position_diff = abs(x1 - x2) + abs(y1 - y2)
    size_diff = abs(w1 - w2) + abs(h1 - h2)
    
    return position_diff < position_threshold and size_diff < size_threshold
    

In [148]:
merged_rois = []
all_rois = []

cap = cv2.VideoCapture(video)

fps = cap.get(cv2.CAP_PROP_FPS)
total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

# Calculate the duration of the video in seconds
video_duration = total_frames / fps

print(f"Video Duration: {video_duration:.2f} seconds")

framelistVideo = []

cv2.namedWindow("Video Frames", cv2.WINDOW_NORMAL)

if not cap.isOpened():
    print("Error: Could not open the video file.")
    exit()

while True:
    ret, frame = cap.read()

    if not ret:
        break

    img = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    rois = extract_blinking_lights(img)
    for roi in rois:
        all_rois.append(roi)
    
    framelistVideo.append(img)

    cv2.imshow("Video Frames", cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the video capture object
cap.release()

# Destroy the OpenCV window
cv2.destroyAllWindows()

if all_rois is not None:
    for roi_coordinates in all_rois:
        add_to_merged = True

        for merged_roi_coordinates in merged_rois:
            if are_similar_coordinates(roi_coordinates, merged_roi_coordinates):
                add_to_merged = False
                break
        
        if add_to_merged:
            merged_rois.append(roi_coordinates)

Video Duration: 3.13 seconds


In [149]:
# Draw the merged ROIs on the frame
for x, y, w, h in merged_rois:
    cv2.rectangle(framelistVideo[4], (x, y), (x + w, y + h), (0, 255, 0), 2)

# Make the frame smaller to fit on the screen
f = cv2.resize(framelistVideo[4], (800, 500))
# Display the frame
cv2.imshow('frame', f)

# Delay to allow time for 'q' key to register
cv2.waitKey(0)
cv2.destroyAllWindows()

In [150]:
rois_in_frames = []

# Create a window for displaying frames
cv2.namedWindow("Video Frames with Bounding Boxes", cv2.WINDOW_NORMAL)

for img in framelistVideo:
    on_off = []
    
    for roi_coordinates in merged_rois:
        x, y, w, h = roi_coordinates
        
        # Ensure the ROI coordinates are within the valid range
        x = x - 30
        y = y - 30
        w = w + 60
        h = h + 60
        
        cv2.rectangle(img, (x, y), (x+w, y+h), (0, 255, 0), 2)
        
        # Extract the ROI
        roi = img[y:y+h, x:x+w]

        # Check if the ROI is not empty before further processing
        if not roi.size == 0:
            # Convert the ROI to grayscale and apply thresholding
            grey = cv2.cvtColor(roi, cv2.COLOR_BGR2GRAY)
            
            # Apply thresholding to highlight the lights
            _, thresholded = cv2.threshold(grey, 170, 255, cv2.THRESH_BINARY)
            
            # Find contours in the thresholded image
            contours, _ = cv2.findContours(thresholded, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

            if len(contours) == 0:
                on_off.append(0)
            else:
                on_off.append(1)
        else:
            on_off.append(0)  # If the ROI is empty, consider it as lights off
    
    rois_in_frames.append(on_off)

    # Display the frame with bounding boxes
    cv2.imshow("Video Frames with Bounding Boxes", cv2.cvtColor(img, cv2.COLOR_BGR2RGB))

    # Introduce a delay (you can adjust the value as needed)
    key = cv2.waitKey(30)

    # Check for the 'q' key to exit the loop
    if key == ord('q'):
        break

# Release the OpenCV window
cv2.destroyAllWindows()

print(rois_in_frames)


[[1, 1, 1, 0], [1, 1, 0, 1], [1, 1, 1, 1], [0, 1, 1, 1], [1, 0, 1, 1], [1, 1, 1, 1], [1, 1, 1, 0], [1, 1, 0, 1], [1, 1, 1, 1], [0, 1, 1, 1], [1, 0, 1, 1], [1, 1, 1, 1], [1, 1, 1, 0], [1, 1, 0, 1], [1, 1, 1, 1], [0, 1, 1, 1], [1, 0, 1, 1], [1, 1, 1, 1], [1, 1, 1, 0], [1, 1, 0, 1], [1, 1, 1, 1], [0, 1, 1, 1], [1, 0, 1, 1], [1, 1, 1, 1], [1, 1, 1, 0], [1, 1, 0, 1], [1, 1, 1, 1], [0, 1, 1, 1], [1, 0, 1, 1], [1, 1, 1, 1], [1, 1, 1, 0], [1, 1, 0, 1], [1, 1, 1, 1], [0, 1, 1, 1], [1, 0, 1, 1], [1, 1, 1, 1], [1, 1, 1, 0], [1, 1, 0, 1], [1, 1, 1, 1], [0, 1, 1, 1], [1, 0, 1, 1], [1, 1, 1, 1], [1, 1, 1, 0], [1, 1, 0, 1], [1, 1, 1, 1], [0, 1, 1, 1], [1, 0, 1, 1], [1, 1, 1, 1], [1, 1, 1, 0], [1, 1, 0, 1], [1, 1, 1, 1], [0, 1, 1, 1], [1, 0, 1, 1], [1, 1, 1, 1], [1, 1, 1, 0], [1, 1, 0, 1], [1, 1, 1, 1], [0, 1, 1, 1], [1, 0, 1, 1], [1, 1, 1, 1], [1, 1, 1, 0], [1, 1, 0, 1], [1, 1, 1, 1], [0, 1, 1, 1], [1, 0, 1, 1], [1, 1, 1, 1], [1, 1, 1, 0], [1, 1, 0, 1], [1, 1, 1, 1], [0, 1, 1, 1], [1, 0, 1, 1], [1, 1

In [151]:
# Transpose the array using zip
cal_freq = []
transposed_array = list(zip(*rois_in_frames))
print(transposed_array)
for i, separated_array in enumerate(transposed_array):
    filtered_tuple = [separated_array[0]] 

    for i in range(1, len(separated_array)):
        if separated_array[i] != separated_array[i - 1]:
            filtered_tuple.append(separated_array[i])
            
    frequency = filtered_tuple.count(1)/video_duration
    print(f"Frequency of light {i}: {frequency}")
    cal_freq.append(frequency)

[(1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0), (1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1), (1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1), (0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0,

In [152]:
print(f'Number of blinking lights: {len(merged_rois)}')

#print all coordinates
for i, coordinates in enumerate(merged_rois):
    print(coordinates)
    #use coordinates to create image roi
    x, y, w, h = coordinates
    #extract from original image
    img = cv2.imread(framelist[0])
    roi = img[y:y+h, x:x+w]
    #display image
    cv2.imshow(f'ROI {i}', roi)

cv2.waitKey(0)
cv2.destroyAllWindows()

Number of blinking lights: 4
(1374, 773, 27, 27)
(458, 283, 27, 27)


(1403, 280, 23, 24)
(504, 824, 23, 22)


In [153]:
import cv2
import numpy as np
from scipy.fft import fft
import concurrent.futures

global previous_frame

# Function to process each ROI
def process_roi(roi):

    x, y, w, h = roi
    x = x - 50
    y = y - 50
    w = w + 100
    h = h + 100
    
    roi_processed = frame[y:y+h, x:x+w]
    roi_processed = cv2.resize(roi_processed, (30, 30), interpolation=cv2.INTER_LINEAR)
    roi_processed = cv2.convertScaleAbs(roi_processed)
    roi_processed = np.array(roi_processed)
    roi_processed = roi_processed / 255.0
    roi_processed = np.expand_dims(roi_processed, axis=0)  # Add batch size dimension

    roi_processed_prev = previous_frame[y:y+h, x:x+w]
    roi_processed_prev = cv2.resize(roi_processed_prev, (30, 30), interpolation=cv2.INTER_LINEAR)
    roi_processed_prev = cv2.convertScaleAbs(roi_processed_prev)
    roi_processed_prev = np.array(roi_processed_prev)
    roi_processed_prev = roi_processed_prev / 255.0
    roi_processed_prev = np.expand_dims(roi_processed_prev, axis=0)  # Add batch size dimension
    
    return np.concatenate((roi_processed, roi_processed_prev))


In [144]:
import contextlib
import io
from datetime import datetime

video_path = './vidscapstone/vid11.mp4'
cap = cv2.VideoCapture(video_path)

if not cap.isOpened():
    print("Error opening video file")
    exit()

fps = cap.get(cv2.CAP_PROP_FPS)  # Get the frames per second from the video

# Initialize roi_frequencies for each ROI
roi_frequencies = {i: [] for i in range(len(merged_rois))}
reset_interval = 90

batch_size = len(merged_rois) * 2
frame_count = 0
frame_total = 0

# Create a window for displaying frames with bounding boxes
cv2.namedWindow("Processed Video", cv2.WINDOW_NORMAL)

timer = cv2.getTickCount()

print("Predictions have been started.")
print(merged_rois)

while True:
    ret, frame = cap.read()

    if not ret:
        break

    frame_count += 1
    frame_total += 1

    # Draw the merged ROIs on the frame

    if frame_count % 2 != 0 and reset_interval != frame_count:
        previous_frame = frame.copy()
        continue

    # Process ROIs sequentially
    rois_processed = [process_roi(roi) for roi in merged_rois]
    
    rois_processed = np.vstack(rois_processed)  # Stack ROIs to create a batch

    # Make predictions on the batch of ROIs
    with contextlib.redirect_stdout(io.StringIO()):
        predictions = model.predict(rois_processed)
        
    predictions = np.where(predictions > 0.5, 1, 0)
    ordered_predictions_1 = []
    ordered_predictions_2 = []
    
    for i, prediction in enumerate(predictions):
        if i % 2 == 0:
            ordered_predictions_1.append(prediction)
        else:
            ordered_predictions_2.append(prediction)

    predictions = np.concatenate((ordered_predictions_1, ordered_predictions_2))
    
    # Draw bounding boxes on the frame
    for i, roi_coordinates in enumerate(merged_rois):
        x, y, w, h = roi_coordinates

        x = x - 50
        y = y - 50
        w = w + 100
        h = h + 100

        roi_index = i % len(merged_rois)

        if predictions[i][0] == 1:
            color = (0, 255, 0)  # Green for lights on
        else:
            color = (0, 0, 255)  # Red for lights off

        cv2.rectangle(frame, (x, y), (x+w, y+h), color, 2)

    # Display the frame with bounding boxes
    cv2.imshow("Processed Video", cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))

    # Introduce a delay (you can adjust the value as needed)
    key = cv2.waitKey(30)

    # Check for the 'q' key to exit the loop
    if key == ord('q'):
        break

    # Distribute predictions to each ROI
    for i, prediction in enumerate(predictions):
        roi_index = i % len(merged_rois)
        if len(roi_frequencies[roi_index]) == 0 or roi_frequencies[roi_index][-1] != prediction[0]:
            roi_frequencies[roi_index].append(prediction[0])

    # Reset frequencies after a certain number of frames
    if frame_count % reset_interval == 0:
        seconds_passed = frame_count / fps
        print(f"Seconds passed: {seconds_passed:.2f}")
        # Calculate the average frequency for each ROI
        for i in range(len(merged_rois)):
            if len(roi_frequencies[i]) > 0:
                frequency = sum(roi_frequencies[i]) / seconds_passed
                print(f"Frequency of light {i}: {frequency:.2f} Hz")
                if frequency < cal_freq[i] - 0.7 or frequency > cal_freq[1] + 0.7:
                    print(f"Inconsistencies on light {i} detected! Please check the light.")
                    print(f"The calculated Frequency is {frequency:.2f} Hz, while the expected frequency is {cal_freq[i]:.2f} Hz.")
                    with open('logs.txt', 'a') as f:
                        f.write(f"Date & Time: {datetime.now().strftime('%d/%m/%Y, %H:%M:%S')} \nWarning: Expected frequency of light {i}: {cal_freq[i]:.2f}\nActual frequency: {frequency:.2f} Hz.\n\n")
                        f.close()

        roi_frequencies = {i: [] for i in range(len(merged_rois))}
        frame_count = 0
        print("Predicting next batch of frames...")

# Calculate FPS
fps = frame_total / ((cv2.getTickCount() - timer) / cv2.getTickFrequency())
print(f"FPS: {fps}")

# Calculate total time
total_time = (cv2.getTickCount() - timer) / cv2.getTickFrequency()
print(f"Total time: {total_time} seconds")
cap.release()

# Release the OpenCV window
cv2.destroyAllWindows()


Predictions have been started.
[(1374, 773, 27, 27), (458, 283, 27, 27), (1403, 280, 23, 24), (504, 824, 23, 22)]
Seconds passed: 3.00
[1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1]
3.0
Frequency of light 0: 5.33 Hz
[1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1]
3.0
Frequency of light 1: 5.00 Hz
[0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1]
3.0
Frequency of light 2: 5.00 Hz
[1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1]
3.0
Frequency of light 3: 5.67 Hz
Predicting next batch of frames...
Seconds passed: 3.00
[1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1]
3.0
Frequency of light 0: 5.33 Hz
[1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1]
3.0
Frequency of light 1: 5.33 Hz
[1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1